In [70]:
import io
import pandas as pd
from datetime import datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import base64
import quandl
from yahoo_fin.stock_info import get_data
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [71]:
def hue_regplot(data, x, y, hue, palette=None, **kwargs):
    from matplotlib.cm import get_cmap

    regplots = []

    levels = data[hue].unique()

    if palette is None:
        default_colors = get_cmap('Dark2')
        palette = {k: default_colors(i) for i, k in enumerate(levels)}

    for key in levels:
        regplots.append(
            sns.regplot(
                x=x,
                y=y,
                data=data[data[hue] == key],
                color=palette[key],
                **kwargs
            ).set(title='CLUSTER ANALYSIS')
        )

    return regplots

In [72]:
def stocks_data(startdate = datetime.strptime('01011990', "%d%m%Y").date(),
                enddate = datetime.strptime('11112020', "%d%m%Y").date()):

    startdate  = startdate.strftime('%Y-%m-%d')
    enddate = enddate.strftime('%Y-%m-%d')
    ### WTI Oil Price
    WTI_daily = quandl.get("FRED/DCOILWTICO",start_date = startdate, end_date = enddate, authtoken="TYxF4cUU1kEsRwg8QEdu", collapse="daily", order="asc")
    ### Exxon Daily Stock Price
    exxon_daily= get_data("XOM", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    ### Chevron Daily Stock Price
    chevron_daily= get_data("CVX", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    ### Conoco Philips Daily Stock Price
    conoco_daily= get_data("COP", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    ### EOG Resources Daily Stock Price
    eog_daily= get_data("EOG", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    ### Valero Energy Daily Stock Price
    valero_daily= get_data("VLO", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    ### Baker Hughes Daily Stock Price
    baker_daily= get_data("BKR", start_date = startdate, end_date = enddate, index_as_date = True, interval="1d")
    daily_data=WTI_daily
    daily_data=daily_data.rename(columns={"Value": "WTI"})
    daily_data=daily_data.merge(exxon_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "exxon_close"})
    daily_data=daily_data.merge(chevron_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "chevron_close"})
    daily_data=daily_data.merge(conoco_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "conoco_close"})
    daily_data=daily_data.merge(eog_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "eog_close"})
    daily_data=daily_data.merge(valero_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "valero_close"})
    daily_data=daily_data.merge(baker_daily[["close"]],how='outer', left_index=True, right_index=True)
    daily_data=daily_data.rename(columns={"close": "baker_close"})
    daily_data.dropna(inplace=True)

    return daily_data

In [73]:
def test_summary(model_target='EOG resources', startdate = datetime.strptime('01011990', "%d%m%Y").date(),
                middate = datetime.strptime('01112019', "%d%m%Y").date(), enddate = datetime.strptime('11112020', "%d%m%Y").date()):
    model_set = {
                "Baker Hughes": "baker_close", "Chevron": "chevron_close", "Conoco Philis": "conoco_close",
                "Exxon Mobile": "exxon_close", "EOG resources": "eog_close", "Valero energy": "valero_close"
                }
    model_x = list(model_set.values())
    model_x.remove(model_set[model_target])
    model_x.append('WTI')

    target_scaled = model_target+'_scaled'
    oil_data = stocks_data(startdate, enddate)
    oil_data.index = pd.to_datetime(oil_data.index).date

    conditions = [
    (oil_data.index <= middate),
    (oil_data.index > middate)
    ]
    values = ['train', 'test']
    oil_data['SPLIT'] = np.select(conditions, values)
    oil_train = oil_data[oil_data['SPLIT'] == 'train']
    oil_test = oil_data[oil_data['SPLIT'] == 'test']
    y_train = oil_train[model_set[model_target]].values.reshape(-1, 1)
    y_test = oil_test[model_set[model_target]].values.reshape(-1, 1)

    X_train = oil_train[model_x]
    X_test = oil_test[model_x]

    # Random Forest model
    regressor = RandomForestRegressor(n_estimators=200, max_depth=5 )

    # Train data
    clf=regressor.fit(X_train, y_train)

    # Predict
    rf_predict =regressor.predict(X_test)

    # LASSO model

    lasso = Lasso(alpha=.01).fit(X_train, y_train)

    lasso_predict = lasso.predict(X_test)
    
    # Output dictionary

    test_date = oil_test.index.to_list()

    model_dict = {}

    for f in range(len(test_date)):      
        model_dict.update({test_date[f].strftime('%m/%d/%Y') : [y_test[f][0], lasso_predict[f], rf_predict[f]]})

    return model_dict


In [75]:
# test_summary()

In [110]:
def lasso_output(model_target='EOG resources', startdate = datetime.strptime('01011990', "%d%m%Y").date(),
                middate = datetime.strptime('01112019', "%d%m%Y").date(), enddate = datetime.strptime('11112020', "%d%m%Y").date()):
    model_set = {
                "Baker Hughes": "baker_close", "Chevron": "chevron_close", "Conoco Philis": "conoco_close",
                "Exxon Mobile": "exxon_close", "EOG resources": "eog_close", "Valero energy": "valero_close"
                }
    model_x = list(model_set.values())
    model_x.remove(model_set[model_target])
    model_x.append('WTI')

    target_scaled = model_target+'_scaled'
    oil_data = stocks_data(startdate, enddate)
    oil_data.index = pd.to_datetime(oil_data.index).date

    conditions = [
    (oil_data.index <= middate),
    (oil_data.index > middate)
    ]
    values = ['train', 'test']
    oil_data['SPLIT'] = np.select(conditions, values)
    oil_train = oil_data[oil_data['SPLIT'] == 'train']
    oil_test = oil_data[oil_data['SPLIT'] == 'test']
    y_train = oil_train[model_set[model_target]].values.reshape(-1, 1)
    y_test = oil_test[model_set[model_target]].values.reshape(-1, 1)

    X_train = oil_train[model_x]
    X_test = oil_test[model_x]

    lasso = Lasso(alpha=.01).fit(X_train, y_train)

    lasso_predict = lasso.predict(X_test)

    lasso_coef = lasso.coef_
    MSE = mean_squared_error(y_test, lasso_predict)
    r2 = lasso.score(X_test, y_test)


    model_dict = { "Mean Square Error": MSE }

    indices=list(X_train)
    for f in range(X_train.shape[1]):
        model_dict.update({indices[f] : lasso_coef[f]})
    
    yTest = oil_test[model_set[model_target]]
    test_date = oil_test.index.to_list()

    model_output = {}

    for f in range(len(test_date)):      
        model_output.update({test_date[f].strftime('%m/%d/%Y') : [y_test[f][0], lasso_predict[f], X_test['WTI'][f]]})

    return sorted(model_dict.items(), key=lambda x: x[1], reverse=True), model_output


In [111]:
lasso_output()

{'11/04/2019': [73.69000244140625, 114.74707846631556, 56.33], '11/05/2019': [74.01000213623047, 114.64272660149054, 57.04], '11/06/2019': [71.41999816894531, 113.02786558521201, 56.15], '11/07/2019': [74.73999786376953, 114.8471729462649, 56.91], '11/08/2019': [74.69999694824219, 114.27769357702708, 57.02], '11/12/2019': [73.29000091552734, 114.44458401715043, 56.67], '11/13/2019': [72.61000061035156, 115.59438727738628, 56.88], '11/14/2019': [72.3499984741211, 115.61740798144477, 56.57], '11/15/2019': [73.4800033569336, 114.19362277721491, 57.54], '11/18/2019': [71.26000213623047, 112.80772410474908, 56.82], '11/19/2019': [70.4000015258789, 109.77304051727414, 54.93], '11/20/2019': [72.05999755859375, 110.2707292864305, 56.71], '11/21/2019': [73.31999969482422, 111.60569504732162, 58.36], '11/22/2019': [72.98999786376953, 111.18393868466622, 57.68], '11/25/2019': [73.5199966430664, 111.03202568734496, 57.79], '11/26/2019': [71.91000366210938, 110.56492012387852, 58.25], '11/27/2019':

[('Mean Square Error', 711.509820399627),
 ('chevron_close', 0.8381967216710241),
 ('valero_close', 0.43966204833901606),
 ('conoco_close', 0.03380470720802007),
 ('WTI', -0.02572563410423235),
 ('baker_close', -0.09793836685957229),
 ('exxon_close', -0.2254154290143795)]